In [1]:
# Google Drive를 Colab에 연결합니다.
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 지리정보 데이터 처리에 필요한 라이브러리를 설치합니다.
!pip install geopandas rasterio


In [5]:
import os
import rasterio
import geopandas as gpd
import numpy as np
import torch
from rasterio.windows import Window
from rasterio.features import rasterize
from shapely.geometry import box

def create_labels_for_colab(
    ortho_image_path,
    building_shp_path,
    output_dir,
    tile_size=1024
):
    """
    지오레퍼런싱된 정사영상과 건물 shapefile을 사용하여
    이미지 타일과 라벨 마스크를 생성하고 저장함
    """

    # 출력 디렉토리 생성
    img_tile_dir = os.path.join(output_dir, "images")
    lbl_tile_dir = os.path.join(output_dir, "labels")
    os.makedirs(img_tile_dir, exist_ok=True)
    os.makedirs(lbl_tile_dir, exist_ok=True)

    # 데이터 로드 (정사영상 및 건물 데이터)
    gdf_buildings = gpd.read_file(building_shp_path)

    with rasterio.open(ortho_image_path) as src:
        # 정사영상 크기를 1024의 배수로 조정
        width = (src.width // tile_size) * tile_size
        height = (src.height // tile_size) * tile_size

        print(f"전체 영상 크기: ({src.width}, {src.height})")
        print(f"1024 배수로 조정된 크기: ({width}, {height})")

        # 건물 데이터를 정사영상 좌표계로 변환 (좌표계가 다를 경우 대비)
        if gdf_buildings.crs != src.crs:
            print(f"건물 데이터의 좌표계({gdf_buildings.crs})를 정사영상({src.crs})에 맞게 변환합니다.")
            gdf_buildings = gdf_buildings.to_crs(src.crs)

        # 타일 단위로 이미지와 라벨 생성
        tile_count = 0
        for j in range(0, height, tile_size):
            for i in range(0, width, tile_size):
                window = Window(i, j, tile_size, tile_size)

                # 현재 타일의 지리적 범위 계산
                tile_transform = src.window_transform(window)
                tile_bounds = rasterio.windows.bounds(window, src.transform)
                tile_bbox = box(*tile_bounds)

                # 타일 범위에 포함되는 건물만 필터링 (효율성 증대)
                intersecting_buildings = gdf_buildings[gdf_buildings.intersects(tile_bbox)]

                if not intersecting_buildings.empty:
                    simple_ids = range(1, len(intersecting_buildings) + 1)
                    shapes = [(geom, id) for geom, id in zip(intersecting_buildings.geometry, simple_ids)]

                    # 이미지 타일 읽기
                    image_tile = src.read(window=window)

                    # 래스터화: 벡터(건물)를 래스터(라벨 마스크)로 변환
                    label_mask = rasterize(
                        shapes=shapes,
                        out_shape=(tile_size, tile_size),
                        transform=tile_transform,
                        fill=0, # 건물이 없는 배경은 0으로 채움
                        all_touched=True,
                        dtype=rasterio.int32
                    )

                    # 정사영상의 밴드가 3개 초과일 경우 RGB(3개)만 사용
                    if image_tile.shape[0] > 3:
                        image_tile = image_tile[:3, :, :]

                    # PyTorch 텐서로 변환하여 저장
                    image_tensor = torch.from_numpy(image_tile).float()
                    label_tensor = torch.from_numpy(label_mask).long() # loss 계산을 위해 long 타입 사용

                    torch.save(image_tensor, os.path.join(img_tile_dir, f"tile_{tile_count}.pt"))
                    torch.save(label_tensor, os.path.join(lbl_tile_dir, f"tile_{tile_count}.pt"))

                    tile_count += 1

        print(f"\n총 {tile_count}개의 타일과 라벨 생성을 완료했습니다.")
        print(f"저장 위치: {output_dir}")

# --- 여기서 경로를 수정가능 ---
# Google Drive에 업로드한 파일들의 경로를 정확하게 입력해야함.

# 1. 1:5000 도엽으로 잘린 정사영상 경로
ORTHO_TIF_PATH = "/content/drive/MyDrive/gis_project/clipped_정사영상.tif"

# 2. 1:5000 도엽으로 잘린 건물 shapefile 경로
BUILDING_SHP_PATH = "/content/drive/MyDrive/gis_project/clipped_buildings.shp"

# 3. 결과물이 저장될 경로 (Google Drive 내)
OUTPUT_DIR = "/content/drive/MyDrive/gis_project/output_tiles"
# ------------------------------------

# 함수 실행
create_labels_for_colab(ORTHO_TIF_PATH, BUILDING_SHP_PATH, OUTPUT_DIR)

전체 영상 크기: (8850, 11114)
1024 배수로 조정된 크기: (8192, 10240)

총 80개의 타일과 라벨 생성을 완료했습니다.
저장 위치: /content/drive/MyDrive/gis_project/output_tiles
